In [23]:
import cv2
import glob
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [24]:
#Step 1: extracting face from the image.
def face_detection(image):
    # Read in the image
    img = mpimg.imread(image)

    # Detect the faces in image
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_img, scaleFactor = 1.3, minNeighbors = 5)
    print(type(faces))
    print(faces)

    for x,y,w,h in faces:
        img = cv2.rectangle(img, (x, y), (x+w, y+h), (255,0,0), 1)

    plt.imshow(img)
    plt.show()

    for x,y,w,h in faces:
        extracted_img = img[y:y+h, x:x+w]
        plt.imshow(extracted_img)
        plt.show()

    return extracted_img

In [25]:
# Step 2: Preprocessing and feature extraction
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors

In [26]:
#Code of K-means from scratch
import numpy as np
import pandas as pd
from sklearn.metrics import silhouette_score, davies_bouldin_score


class KMeans:
    def __init__(self, k, max_iter=5, init_method="kmeans++"):
        self.k = k
        self.max_iter = max_iter
        self.inertia = np.inf
        self.init_method = init_method

    def init_centroids(self, X, num_centroids=None):
        if self.init_method == "forgy":
            self.init_forgy(X, num_centroids)
        elif self.init_method == "kmeans++":
            self.init_kmeans(X, num_centroids)

    def init_forgy(self, X, num_centroids=None):
        # "Forgy" initialization method
        if num_centroids is None:
            num_centroids = self.k
        assert (
            num_centroids <= X.shape[0]
        ), "Number of centroids must be less than number of data points"
        rand_indices = np.random.choice(X.shape[0], num_centroids, replace=False)
        self.centroids = X[rand_indices]
        print(self.centroids)

    def init_kmeans(self, X, num_centroids=None):
        # "kmeans++" initialization method
        if num_centroids is None:
            num_centroids = self.k
        assert (
            num_centroids <= X.shape[0]
        ), "Number of centroids must be less than number of data points"
        self.centroids = np.zeros((num_centroids, X.shape[1]))
        self.centroids[0] = X[np.random.choice(X.shape[0], 1, replace=False)]
        for i in range(1, num_centroids):
            dists = np.linalg.norm(X - self.centroids[i - 1], axis=1)
            probs = dists / np.sum(dists)
            self.centroids[i] = X[
                np.random.choice(X.shape[0], 1, replace=False, p=probs)
            ]

    def calc_inertia(self, X):
        return sum(
            np.linalg.norm(X[i] - self.centroids[int(self.labels[i])]) ** 2
            for i in range(X.shape[0])
        )

    def fit(self, X):
        self.init_centroids(X)
        self.labels = np.zeros(X.shape[0])
        self.iter = 0
        print("-" * 80)
        print("Initial inertia: ", self.calc_inertia(X))

        # Convergence criteria 1: Number of iterations is less than max_iter
        while self.iter < self.max_iter:

            # Assignment step for each data point
            for i in range(X.shape[0]):
                self.labels[i] = np.argmin(
                    np.linalg.norm(X[i] - self.centroids, axis=1)
                )

            # Update step for each centroid
            for i in range(self.k):
                self.centroids[i] = np.mean(X[self.labels == i], axis=0)

            self.iter += 1

            # Convergence criteria 2: Inertia should be less than previous inertia
            if self.calc_inertia(X) >= self.inertia:
                break

            self.inertia = self.calc_inertia(X)
            self.metric1 = davies_bouldin_score(X, self.labels)
            self.metric2 = silhouette_score(X, self.labels)
            print("-" * 80)
            print("Iteration: ", self.iter)
            print("Inertia: ", self.inertia)
            print("Sklearn metrics : ")
            print("Silhouette score: ", self.metric2)
            print("Davies-Bouldin score: ", self.metric1)

        # Calculate inertia and homogeneous after convergence
        self.inertia = self.calc_inertia(X)
        self.metric1 = davies_bouldin_score(X, self.labels)
        self.metric2 = silhouette_score(X, self.labels)
        print("-" * 80)
        print("Final inertia: ", self.inertia)
        print("Sklearn metrics : ")
        print("Silhouette score: ", self.metric2)
        print("Davies-Bouldin score: ", self.metric1)
        print("-" * 80)

    def predict(self, X):
        # return the labels for each data point in X
        labels = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            labels[i] = np.argmin(np.linalg.norm(X[i] - self.centroids, axis=1))
        labels = [int(label + 1) for label in labels]
        return labels


In [27]:
# Step 3: Clustering
def cluster_features(features, num_clusters):
    features_array = np.concatenate(features)  # Convert list of arrays to a single NumPy array
    kmeans = KMeans(k=num_clusters)
    kmeans.fit(features_array.astype('float32'))  # Convert features to float32
    return kmeans

In [28]:
# Step 4: Assigning visual words
def assign_visual_words(features, kmeans):
    visual_words = kmeans.predict(features)
    visual_words_hist, _ = np.histogram(visual_words, bins=range(kmeans.n_clusters + 1), density=True)
    return visual_words_hist.astype('float32')

In [29]:
# Step 5: Encoding
def encode_image(image_path, kmeans):
    features = preprocess_image(image_path)
    if features is not None:
        visual_words = assign_visual_words(features, kmeans)
        return visual_words
    return None

In [30]:
# Step 6: Training
def train_svm(train_data, train_labels):
    svm = LinearSVC()
    svm.fit(train_data, train_labels)
    return svm

In [31]:
#step 7: classification
def classify_image(image_path, svm, kmeans):
    encoded_image = encode_image(image_path, kmeans)
    if encoded_image is not None:
        label = svm.predict([encoded_image])
        return label[0]
    return None

In [32]:

# Example usage
# Train a classifier using the training dataset
train_data = []
train_labels = []

classes = sorted(glob.glob("images/train/*"))
print(classes)

['images/train/angry', 'images/train/happy', 'images/train/sad', 'images/train/surprise']


In [33]:
for i in range(len(classes)):
    if i == 0:
        # Get the list of image paths for the current class
        class_images = glob.glob(classes[i] + "/*.jpg")
        num_images_per_class = len(class_images)

        features = []
        for j in range(num_images_per_class):
            # Process each image and extract its descriptors
            image_path = class_images[j]
            descriptors = preprocess_image(image_path)
            if descriptors is not None:
                features.append(descriptors)

        if len(features) > 0:
            # Cluster the extracted features using K-means
            kmeans = cluster_features(features, num_clusters=100)
            for j in range(num_images_per_class):
                # Encode each image using the K-means clusters
                image_path = class_images[j]
                encoded_image = encode_image(image_path, kmeans)
                if encoded_image is not None:
                    # Append the encoded image and its label to the training data
                    train_data.append(encoded_image.astype('float32'))  # Convert encoded_image to float32
                    train_labels.append(i)

--------------------------------------------------------------------------------
Initial inertia:  29961750638.0
--------------------------------------------------------------------------------
Iteration:  1
Inertia:  9959530377.667269
Sklearn metrics : 
Silhouette score:  0.013066542
Davies-Bouldin score:  3.5593630570050085
--------------------------------------------------------------------------------
Iteration:  2
Inertia:  9436754858.903145
Sklearn metrics : 
Silhouette score:  0.026699942
Davies-Bouldin score:  3.379272787812208
--------------------------------------------------------------------------------
Iteration:  3
Inertia:  9287008675.207102
Sklearn metrics : 
Silhouette score:  0.028982399
Davies-Bouldin score:  3.2770824536870857
--------------------------------------------------------------------------------
Iteration:  4
Inertia:  9212625051.82595
Sklearn metrics : 
Silhouette score:  0.029852621
Davies-Bouldin score:  3.2160382446753233
-----------------------------

AttributeError: 'KMeans' object has no attribute 'n_clusters'

In [ ]:
svm = train_svm(train_data, train_labels)

In [ ]:
extracted_face = face_detection('sad.jpg')
filename = "face.jpg"
cv2.imwrite(filename, extracted_face)

In [ ]:
test_image_path = "face.jpg"
predicted_label = classify_image(test_image_path, svm, kmeans)
if predicted_label is not None:
    print("Predicted label:", predicted_label)
else:
    print("Unable to classify the image.")